<a href="https://colab.research.google.com/github/bbygsha/Shawna-Daniels-JSURE-2021/blob/Python-Codes-and-Data/FOPDTmethodShawnaDaniels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Order Plus Dead Time (FOPDT)
This approach is using optimal control dynamics. See the link below. 
https://apmonitor.com/pdc/index.php/Main/FirstOrderSystems


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 29 19:49:53 2021

@author: Widodo Samyono
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.interpolate import interp1d

# Import CSV data file
# Column 1 = time (t)
# Column 2 = input (u)
# Column 3 = output (yp)
#upload from Google drive
from google.colab import drive
drive.mount('/content/drive')
# url = 'http://apmonitor.com/pdc/uploads/Main/data_fopdt.txt'
# data = pd.read_csv(url)
#data = pd.read_csv("dataskylert20.txt")
#data = pd.read_csv("datajordan.txt")
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/growthcurveMCF7pd.txt")
# print(data)
t = data['time'].values - data['time'].values[0]
# print(t)
u = data['y'].values
yp = data['y'].values
u0 = yp[0]
yp0 = yp[0]

# specify number of steps
ns = len(t)
delta_t = t[1]-t[0]
# create linear interpolation of the u data versus time
uf = interp1d(t,u)

# define first-order plus dead-time approximation    
def fopdt(y,t,uf,Km,taum,thetam):
    # arguments
    #  y      = output
    #  t      = time
    #  uf     = input linear function (for time shift)
    #  Km     = model gain
    #  taum   = model time constant
    #  thetam = model time constant
    # time-shift u
    try:
        if (t-thetam) <= 0:
            um = uf(0.0)
        else:
            um = uf(t-thetam)
    except:
        #print('Error with time extrapolation: ' + str(t))
        um = u0
    # calculate derivative
    dydt = (-(y-yp0) + Km * (um-u0))/taum
    return dydt

# simulate FOPDT model with x=[Km,taum,thetam]
def sim_model(x):
    # input arguments
    Km = x[0]
    taum = x[1]
    thetam = x[2]
    # storage for model values
    ym = np.zeros(ns)  # model
    # initial condition
    ym[0] = yp0
    # loop through time steps    
    for i in range(0,ns-1):
        ts = [t[i],t[i+1]]
        y1 = odeint(fopdt,ym[i],ts,args=(uf,Km,taum,thetam))
        ym[i+1] = y1[-1]
    return ym

# define objective
def objective(x):
    # simulate model
    ym = sim_model(x)
    # calculate objective
    obj = 0.0
    for i in range(len(ym)):
        obj = obj + (ym[i]-yp[i])**2    
    # return result
    return obj

# initial guesses
x0 = np.zeros(3)
x0[0] = 1.0      # Km     = 1.0
x0[1] = 0.9      # taum   = 0.005, 0.5, 0.9
x0[2] = 0.0      # thetam = 0.0
#        SSE    9.035281498e-05, 1.271059134e+05, 2.627754202e+05              
# show initial objective
print('Initial SSE Objective: ' + "{:.9e}".format((objective(x0))))
print('Kp0: ' + str(x0[0]))
print('taup0: ' + str(x0[1]))
print('thetap0: ' + str(x0[2]))


# optimize Km, taum, thetam
# solution = minimize(objective,x0)
solution = minimize(objective,x0,method='powell',
              options={'xtol': 1e-8, 'maxfev':10000,'disp': True})
# Another way to solve: with bounds on variables
#bnds = ((0.4, 0.6), (1.0, 10.0), (0.0, 30.0))
#solution = minimize(objective,x0,bounds=bnds,method='SLSQP')
x = solution.x

# show final objective
print('Final SSE Objective: ' + "{:.9e}".format((objective(x))))

print('Kp: ' + str(x[0]))
print('taup: ' + str(x[1]))
print('thetap: ' + str(x[2]))

# calculate model with updated parameters
ym1 = sim_model(x0)
ym2 = sim_model(x)
# plot results
plt.figure()
plt.subplot(2,1,1)
plt.plot(t,yp,'kx-',linewidth=2,label='Process Data')
plt.plot(t,ym1,'b-',linewidth=2,label='Initial Guess')
plt.plot(t,ym2,'r--',linewidth=3,label='Optimized FOPDT')
plt.ylabel('Output')
plt.legend(loc='best')
plt.subplot(2,1,2)
plt.plot(t,u,'bx-',linewidth=2)
plt.plot(t,uf(t),'r--',linewidth=3)
plt.legend(['Measured','Interpolated'],loc='best')
plt.ylabel('Input Data')
plt.show()


Mounted at /content/drive


FileNotFoundError: ignored